# Number of lines in file

In [17]:
data_file = '/Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt'
count = 0
for line in open(data_file): count += 1

In [18]:
count

577335302

In [6]:
%%bash
head -2 /Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt

GLOBAL UNIQUE IDENTIFIER	LAST EDITED DATE	TAXONOMIC ORDER	CATEGORY	COMMON NAME	SCIENTIFIC NAME	SUBSPECIES COMMON NAME	SUBSPECIES SCIENTIFIC NAME	OBSERVATION COUNT	BREEDING BIRD ATLAS CODE	BREEDING BIRD ATLAS CATEGORY	AGE/SEX	COUNTRY	COUNTRY CODE	STATE	STATE CODE	COUNTY	COUNTY CODE	IBA CODE	BCR CODE	USFWS CODE	ATLAS BLOCK	LOCALITY	LOCALITY ID	 LOCALITY TYPE	LATITUDE	LONGITUDE	OBSERVATION DATE	TIME OBSERVATIONS STARTED	OBSERVER ID	SAMPLING EVENT IDENTIFIER	PROTOCOL TYPE	PROTOCOL CODE	PROJECT CODE	DURATION MINUTES	EFFORT DISTANCE KM	EFFORT AREA HA	NUMBER OBSERVERS	ALL SPECIES REPORTED	GROUP IDENTIFIER	HAS MEDIA	APPROVED	REVIEWED	REASON	TRIP COMMENTS	SPECIES COMMENTS
URN:CornellLabOfOrnithology:EBIRD:OBS34169564	2014-10-16 08:19:55	6171	species	Bonaparte's Gull	Chroicocephalus philadelphia			6				Canada	CA	Quebec	CA-QC	Nicolet-Yamaska	CA-QC-NY					Port St-François	L352699	P	46.2722222	-72.6111111	2006-10-14	10:00:00	obsr31597	S2495584	Traveling	P22	EBIRD_CAN	60	1		1	0		0	1	0		Terns are lat

# Store data in a database

* Determine which columns to store
* Store a small subset of text file
* Practice looking into subset DB
* Do chaser on subset
* Store entire DB (or just USA, Canada?)
* Do chaser 

In [ ]:
GLOBAL UNIQUE IDENTIFIER	                    LAST EDITED DATE	        TAXONOMIC ORDER	    CATEGORY	    COMMON NAME	        SCIENTIFIC NAME	               SUBSPECIES COMMON NAME	SUBSPECIES SCIENTIFIC NAME	OBSERVATION COUNT	BREEDING BIRD ATLAS CODE	BREEDING BIRD ATLAS CATEGORY	AGE/SEX	COUNTRY	COUNTRY CODE	STATE	STATE CODE	COUNTY	            COUNTY CODE	IBA CODE	BCR CODE	USFWS CODE	ATLAS BLOCK	LOCALITY	        LOCALITY ID	LOCALITY TYPE	LATITUDE	LONGITUDE	OBSERVATION DATE	TIME OBSERVATIONS STARTED	OBSERVER ID	SAMPLING EVENT IDENTIFIER	PROTOCOL TYPE	PROTOCOL CODE	PROJECT CODE	DURATION MINUTES	EFFORT DISTANCE KM	EFFORT AREA HA	NUMBER OBSERVERS	ALL SPECIES REPORTED	GROUP IDENTIFIER	HAS MEDIA	APPROVED	REVIEWED	REASON	TRIP COMMENTS	SPECIES COMMENTS
URN:CornellLabOfOrnithology:EBIRD:OBS34169564	2014-10-16 08:19:55	        6171	            species	        Bonaparte's Gull	Chroicocephalus philadelphia		                    	                        6	                	                        	                        	        Canada	CA	            Quebec	CA-QC	     Nicolet-Yamaska	CA-QC-NY		        	        	        	        Port St-François	L352699	    P	            46.2722222	-72.6111111	2006-10-14	         10:00:00	                obsr31597	S2495584	                Traveling	    P22	            EBIRD_CAN	    60	                1	                	            1	                0	                    	                0	        1	        0	        	    Terns are late!	
                
NEEDED ROWS:
Last edited date, category, common name, scientific name, observation count, country, state, latitude, longitude, observation date, observer id, protocol type
                    
                    

In [1]:
import pandas as pd

In [2]:
import time

In [3]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database


In [51]:
engine2 = create_engine("sqlite:////Volumes/seagate-storage/db/ebird_2018_small.db")
if not database_exists(engine.url):
    create_database(engine.url)

In [58]:
data_file = '/Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt'
#data_file = '/Volumes/seagate-storage/code/ebird_data/medium_dataset.txt'

In [59]:
num_lines = 577335302 #from previous analysis
chunk_size = 1000000 # number of lines to read at once
num_chunks = 3 # number of chunks to test on

In [47]:
the_cols = [
        'Last edited date', 
        'category', 
        'common name', 
        'scientific name', 
        'observation count', 
        'country', 
        'state', 
        'latitude', 
        'longitude', 
        'observation date', 
        'observer id', 
        'protocol type',
        'ALL SPECIES REPORTED',
]

desired_cols = [item.upper() for item in the_cols]
desired_cols

['LAST EDITED DATE',
 'CATEGORY',
 'COMMON NAME',
 'SCIENTIFIC NAME',
 'OBSERVATION COUNT',
 'COUNTRY',
 'STATE',
 'LATITUDE',
 'LONGITUDE',
 'OBSERVATION DATE',
 'OBSERVER ID',
 'PROTOCOL TYPE',
 'ALL SPECIES REPORTED']

In [44]:
just_byemeh = pd.DataFrame({'hi':[0, 1, 2], 'bye':[8, 9, 9], 'meh':[42, 42, 20]}, index=[0, 1, 2])[['bye', 'meh']]
just_byemeh.loc[just_byemeh['meh'] == 42]

just_byemeh.loc[
    (just_byemeh['meh'] == 42) & 
    (just_byemeh['bye'] == 9)]

,bye,meh
1,9,42


In [77]:

engine = create_engine("sqlite:////Volumes/seagate-storage/db/ebird_2018.db")
if not database_exists(engine.url):
    create_database(engine.url)
    
    
    
the_cols = [
        'Last edited date', 
        'category', 
        'common name', 
        'scientific name', 
        'observation count', 
        'country', 
        'state', 
        'latitude', 
        'longitude', 
        'observation date', 
        'observer id', 
        'protocol type',
        'ALL SPECIES REPORTED',
]

desired_cols = [item.upper() for item in the_cols]
desired_cols


data_file = '/Volumes/seagate-storage/code/evil_project/example_dataset.txt' #'/Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt'


num_lines = 577335302 #from previous analysis
chunk_size = 1000000 # number of lines to read at once


def process(chunk):
    
    only_2018 = chunk.loc[
        (chunk['LAST EDITED DATE'].dt.year == 2018) &
        (chunk['OBSERVATION DATE'].dt.year == 2018)]
    
    if only_2018.shape[0]: # if this DF has anything in it
        only_2018.to_sql('ebird_data_2018', engine, if_exists='append')

    return only_2018

t1 = time.time()

lines_saved_to_db = 0
counter = 0

for chunk in pd.read_csv(
    data_file, 
    sep='\t', 
    #index_col = ['SAMPLING EVENT IDENTIFIER'],
    usecols = desired_cols,
    parse_dates = ['LAST EDITED DATE', 'OBSERVATION DATE'], 
    chunksize=chunk_size):
    
    '''
    '''
    
    finished_chunk = process(chunk)
    lines_saved_to_db += finished_chunk.shape[0]

    if not counter % 100:
        print(f"Iteration {counter}:")
        print(f"  Lines saved to DB: {lines_saved_to_db}")
        print(f"  Minutes elapsed: {(time.time()-t1)/60}")
    counter += 1
    
    
t2 = time.time()

process_time = t2-t1
print('Time to process rows:', process_time)

#est_time = (num_lines/(chunk_size*num_chunks))*process_time
#print('Estimated time (seconds) for all rows:', est_time)

Iteration 0:
  Lines saved to DB: 3
  Minutes elapsed: 0.005988717079162598
Time to process rows: 0.3595271110534668


Reading the whole DF:

```
chunk_size = 1000000 # number of lines to read at once
num_chunks = 3 # number of chunks to test on

(1000000, 46)
(1000000, 46)
(1000000, 46)
Time to process 3 rows: 23.173866748809814
Estimated time for all rows: 4459.697119310624


---

chunk_size = 3000000 # number of lines to read at once
num_chunks = 1 # number of chunks to test on

(3000000, 46)
Time to process rows: 23.521601915359497
Estimated time (seconds) for all rows: 4526.6170484426175

---

chunk_size = 500000 # number of lines to read at once
num_chunks = 6 # number of chunks to test on

(500000, 46)
(500000, 46)
(500000, 46)
(500000, 46)
(500000, 46)
(500000, 46)
Time to process rows: 24.359390020370483
Estimated time (seconds) for all rows: 4687.845264648793
```

Reading only desired columns, with `parse_dates`
```
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
Time to process rows: 13.444914102554321
Estimated time (seconds) for all rows: 2587.407847920753
```

Reading only desired columns, without `parse_dates`:

```
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
Time to process rows: 11.651726961135864
Estimated time (seconds) for all rows: 2242.3177679763053
```


In [ ]:
pd.read_csv(small_data_file, sep='\t').to_sql('test_ebird_data', engine)

In [40]:
import pandas as pd
small_data_file = '/Volumes/seagate-storage/code/ebird_data/test_dataset.txt'
pd.read_csv(small_data_file, sep='\t').to_sql('test_ebird_data', engine)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-088db2b34f23>", line 3, in <module>
    pd.read_csv(small_data_file, sep='\t').to_sql('test_ebird_data', engine)
  File "/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py", line 1534, in to_sql
    chunksize=chunksize, dtype=dtype)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/sql.py", line 473, in to_sql
    chunksize=chunksize, dtype=dtype)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/sql.py", line 1155, in to_sql
    table.create()
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/sql.py", line 592, in create
    raise ValueError("Table '%s' already exists." % self.name)
ValueError: Table 'test_ebird_data' already exists.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):


ValueError: Table 'test_ebird_data' already exists.

# Clean up data

In [54]:
import time
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import pandas as pd
from functools import wraps
from time import time

In [ ]:
small_db = pd.read_sql('test_ebird_data',con=engine)

In [42]:
small_db.columns

Index(['index', 'GLOBAL UNIQUE IDENTIFIER', 'LAST EDITED DATE',
       'TAXONOMIC ORDER', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME',
       'SUBSPECIES COMMON NAME', 'SUBSPECIES SCIENTIFIC NAME',
       'OBSERVATION COUNT', 'BREEDING BIRD ATLAS CODE',
       'BREEDING BIRD ATLAS CATEGORY', 'AGE/SEX', 'COUNTRY', 'COUNTRY CODE',
       'STATE', 'STATE CODE', 'COUNTY', 'COUNTY CODE', 'IBA CODE', 'BCR CODE',
       'USFWS CODE', 'ATLAS BLOCK', 'LOCALITY', 'LOCALITY ID',
       ' LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
       'TIME OBSERVATIONS STARTED', 'OBSERVER ID', 'SAMPLING EVENT IDENTIFIER',
       'PROTOCOL TYPE', 'PROTOCOL CODE', 'PROJECT CODE', 'DURATION MINUTES',
       'EFFORT DISTANCE KM', 'EFFORT AREA HA', 'NUMBER OBSERVERS',
       'ALL SPECIES REPORTED', 'GROUP IDENTIFIER', 'HAS MEDIA', 'APPROVED',
       'REVIEWED', 'REASON', 'TRIP COMMENTS', 'SPECIES COMMENTS'],
      dtype='object')

### Keep only the needed columns

In [59]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func took: {} sec'.format(te-ts))
        return result
    return wrap


def process_chunk(chunk_df):
    new_df = chunk_df[['COMMON NAME','LATITUDE', 'LONGITUDE', 'OBSERVATION DATE']]
    new_df.to_sql('med_occ', medium_engine, if_exists='append')

@timing
def process_all_data(engine_path, data_path):
    engine = create_engine('sqlite:///'+engine_path)
    for chunk in pd.read_csv(data_path, sep='\t', chunksize=100):
        process_chunk(chunk)


process_all_data(
    engine_path='/Volumes/seagate-storage/db/large_ebird.db', 
    data_path='/Volumes/seagate-storage/code/ebird_data/large_dataset.txt')

func took: 17.197799921035767 sec


In [43]:
small_database = database[['COMMON NAME','LATITUDE', 'LONGITUDE', 'OBSERVATION DATE']]
small_database

,COMMON NAME,LATITUDE,LONGITUDE,OBSERVATION DATE
0,Bonaparte's Gull,46.272222,-72.611111,2006-10-14
1,Long-tailed Duck,43.468586,-76.507357,2008-01-06
2,Herring Gull,43.468586,-76.507357,2008-01-06
3,American Robin,49.307119,-123.004286,2008-04-05
4,Stewart Island Shag,-45.797810,170.714986,2008-01-25
5,Silver-eye,-45.797810,170.714986,2008-01-25
6,New Zealand Bellbird,-45.797810,170.714986,2008-01-25
7,Yellowhammer,-45.797810,170.714986,2008-01-25
8,Little Pied Cormorant,-45.797810,170.714986,2008-01-25
